In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

In [ ]:
print("Initial shape:", df.shape)

In [ ]:
df.info()

In [ ]:
# Drop irrelevant columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
# Encode categorical variables
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])

le_geo = LabelEncoder()
df['Geography'] = le_geo.fit_transform(df['Geography'])

In [ ]:
df.head()

In [ ]:
# Split features and target
X = df.drop('Exited', axis=1)
y = df['Exited']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def build_and_train_nn(X_train, y_train, X_test, y_test, 
                       hidden_layers=[64,32], activation='relu', epochs=50):
    # Initialize model
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_dim=X_train.shape[1], activation=activation))
    
    # Hidden layers
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    
    # Compile
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=32, verbose=0)
    
    # Predictions
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    print(f"Activation: {activation}, Epochs: {epochs}")
    print(f"Accuracy: {acc:.3f}")
    print("Confusion Matrix:\n", cm)
    
    return model, history


In [ ]:
# Experiment with different activations and epochs
activations = ['relu', 'tanh', 'sigmoid']
epochs_list = [25, 50, 75]

results = []

for act in activations:
    for ep in epochs_list:
        model, history = build_and_train_nn(X_train_scaled, y_train, X_test_scaled, y_test,
                                            activation=act, epochs=ep)
        results.append((act, ep, model, history))

In [ ]:
import matplotlib.pyplot as plt

# One subplot per activation function
activations = sorted(list(set([r[0] for r in results])))
epochs_list = sorted(list(set([r[1] for r in results])))

plt.figure(figsize=(12, 6))

for i, act in enumerate(activations):
    plt.subplot(1, len(activations), i+1)
    
    for act_r, ep, model, history in results:
        if act_r == act:
            plt.plot(history.history['val_accuracy'], label=f'{ep} epochs')
    
    plt.title(f'Validation Accuracy - {act}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.legend()

plt.tight_layout()
plt.show()